### **Code : (1) 금융 데이터 매칭 패턴 모델링**

- Project : 2024 데이터바우처 지원사업
- Writer : Donghyeon Kim
- Update : 2024.10.05.

#### **0. 라이브러리 및 초기 경로 설정**

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from tqdm import tqdm

In [4]:
dr = 'C:/'
folder_1 = 'Users/USER/Dropbox/8. 회사업무/1. 산학협력프로젝트/2024년/9. [선정] 2024 데이터바우처 지원사업(경리법인일조)/7. 분석'
root = dr + folder_1

folder_2 = '1_rawdata'
folder_path = root + '/' + folder_2

os.chdir(folder_path)

In [5]:
# 회사이름
company_name = os.listdir()[6] # Available Index : 0 ~ 9 (Total : 10)

# 회사이름 포함 경로
final_path = folder_path + '/' + company_name
os.chdir(final_path)

---

#### **1. 입금내역 Data Load**

In [16]:
# '입금내역_수납확인' 파일명
file_name = [file for file in os.listdir() if '입금내역_수납확인' in file] # 입금내역 파일 1개

# Data Frame
df = pd.read_excel(file_name[0], skiprows=1)

# Data Frame Head(2023)
df_without_account = df.drop(columns=['계좌번호', '계좌적요', '거래처'])
df_without_account[df_without_account['입금일자'].str.contains('2023-12')].head()

,입금일자,은행,입금액,용도,내용,프로젝트/현장,비고,메모
105,2023-12-29,우리은행,660000,매출대금입금,NaN,NaN,2023-11-23 세계2023-12-28 세계,NaN
106,2023-12-28,우리은행,330000,매출대금입금,NaN,NaN,2023-12-20 세계,NaN
107,2023-12-28,우리은행,2607000,매출대금입금,NaN,NaN,2023-11-30 세계,NaN
108,2023-12-28,우리은행,2200000,매출대금입금,NaN,NaN,2023-12-27 세계,NaN
109,2023-12-19,우리은행,440000,매출대금입금,NaN,NaN,2023-12-19 세계,NaN


#### **1) 입금 파일 : '용도' Prediction**

In [17]:
# '입금일자'를 datetime 형식으로 변환하고 2023년까지 필터링
df['입금일자'] = pd.to_datetime(df['입금일자'], errors='coerce')
df_filtered = df[df['입금일자'].dt.year <= 2023].copy()

# '용도'와 '거래처'의 결측치가 있는 행 제거
df_filtered_cleaned = df_filtered.dropna(subset=['용도', '거래처']).copy()

# 예측에 사용할 특성 선택
df_features = df_filtered_cleaned[['입금액', '계좌적요', '은행', '용도', '거래처']].copy()

# 결측치가 있는 행 제거
df_features_cleaned = df_features.dropna().copy()

# 클래스별 데이터 개수 확인
class_counts = df_features_cleaned['용도'].value_counts()

# 클래스 수가 2개 미만인 클래스 제거
min_class_count = 2
classes_to_keep = class_counts[class_counts >= min_class_count].index
df_filtered_cleaned = df_features_cleaned[df_features_cleaned['용도'].isin(classes_to_keep)].copy()

# 다시 X와 y 정의
X = df_filtered_cleaned[['입금액', '계좌적요', '은행']].copy()
y = df_filtered_cleaned['용도'].copy()

# 범주형 변수(Label Encoding) 변환 (클래스 제거 후 적용)
label_encoder_계좌적요 = LabelEncoder()
label_encoder_은행 = LabelEncoder()
label_encoder_용도 = LabelEncoder()

# Fit Transform
X['계좌적요'] = label_encoder_계좌적요.fit_transform(X['계좌적요'].copy())
X['은행'] = label_encoder_은행.fit_transform(X['은행'].copy())
y = label_encoder_용도.fit_transform(y.copy())

# 데이터셋을 stratify를 사용해 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 클래스 불균형 문제를 해결하기 위해 Random Over Sampling 적용
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 오버샘플링한 학습 데이터에서 20%를 검증 데이터로 분리 (eval_set용)
X_train_ros, X_val_ros, y_train_ros, y_val_ros = train_test_split(
    X_train_ros, y_train_ros, test_size=0.2, random_state=42, stratify=y_train_ros
)

# XGBoost 모델 학습 (eval_set을 사용해 성능 모니터링)
xgb_model = XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y_train)), eval_metric='mlogloss')

print('XGBoost Training Progress:')
xgb_model.fit(X_train_ros, y_train_ros, eval_set=[(X_val_ros, y_val_ros)])

# RandomForest 모델 학습 시 tqdm 진행률 표시
n_estimators = 100
rf_model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

print('RandomForest Training Progress:')
for i in tqdm(range(1, n_estimators + 1), desc='RandomForest Training'):
    rf_model.set_params(n_estimators=i)
    rf_model.fit(X_train_ros, y_train_ros)

# Voting Classifier 생성 (XGBoost + RandomForest)
voting_clf = VotingClassifier(estimators=[('xgb', xgb_model), ('rf', rf_model)], voting='soft')

# Voting Classifier 학습 시 tqdm 진행률 표시
print('Voting Classifier Training Progress:')
for _ in tqdm(range(1), desc='Voting Classifier Training'):
    voting_clf.fit(X_train_ros, y_train_ros)

# 테스트 데이터로 예측 수행
y_pred = voting_clf.predict(X_test)

XGBoost Training Progress:
[0]	validation_0-mlogloss:0.99415
[1]	validation_0-mlogloss:0.69669
[2]	validation_0-mlogloss:0.51397
[3]	validation_0-mlogloss:0.38901
[4]	validation_0-mlogloss:0.29362


[5]	validation_0-mlogloss:0.22794
[6]	validation_0-mlogloss:0.17536
[7]	validation_0-mlogloss:0.13705
[8]	validation_0-mlogloss:0.10929
[9]	validation_0-mlogloss:0.08871
[10]	validation_0-mlogloss:0.07286
[11]	validation_0-mlogloss:0.06073
[12]	validation_0-mlogloss:0.05089
[13]	validation_0-mlogloss:0.04325
[14]	validation_0-mlogloss:0.03792
[15]	validation_0-mlogloss:0.03386
[16]	validation_0-mlogloss:0.03062
[17]	validation_0-mlogloss:0.02811
[18]	validation_0-mlogloss:0.02611
[19]	validation_0-mlogloss:0.02495
[20]	validation_0-mlogloss:0.02398
[21]	validation_0-mlogloss:0.02315
[22]	validation_0-mlogloss:0.02266
[23]	validation_0-mlogloss:0.02231
[24]	validation_0-mlogloss:0.02202
[25]	validation_0-mlogloss:0.02188
[26]	validation_0-mlogloss:0.02179
[27]	validation_0-mlogloss:0.02177
[28]	validation_0-mlogloss:0.02179
[29]	validation_0-mlogloss:0.02182
[30]	validation_0-mlogloss:0.02186
[31]	validation_0-mlogloss:0.02191
[32]	validation_0-mlogloss:0.02197
[33]	validation_0-mloglos

RandomForest Training: 100%|██████████| 100/100 [00:11<00:00,  8.70it/s]


Voting Classifier Training Progress:


Voting Classifier Training: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]


In [19]:
# XGBoost + RandomForest Ensemble Result
# LabelEncoder로 예측된 값을 다시 원래 값으로 디코딩
y_test_original = label_encoder_용도.inverse_transform(y_test)
y_pred_original = label_encoder_용도.inverse_transform(y_pred)

# Classification_report를 Dictionary로 변환
report = classification_report(y_test_original, y_pred_original, zero_division=1, output_dict=True)

# Accuracy만 출력
accuracy = round(report['accuracy'], 3)
print(f"Accuracy: {accuracy}")

Accuracy: 0.985


In [21]:
# classification_report를 데이터프레임으로 변환하고 엑셀로 저장하는 간단한 코드
report_df = pd.DataFrame(report).transpose()

# 소수점 3자리로 변환
report_df = report_df.round(3)

# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(1st)')
company_result_root = os.path.join(result_root, company_name)
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

# 결과물 최종 경로
output_file = os.path.join(company_result_root, f'{company_name}_입금내역_용도.xlsx')

# 엑셀 파일로 저장
report_df.to_excel(output_file, index=True)
print('Classification Report가 Excel 파일로 저장되었습니다.')

Classification Report가 Excel 파일로 저장되었습니다.


In [22]:
# 2024년 Data 예측 #

# '입금일자'를 datetime 형식으로 변환하고 2024년 필터링
df_2024 = df[df['입금일자'].dt.year == 2024].copy()

# 예측에 필요한 컬럼만 선택
df_2024_features = df_2024[['입금액', '계좌적요', '은행']].copy()

# 결측치가 있는 행 제거 (원본 데이터에서 인덱스 추적)
df_2024_features_cleaned = df_2024_features.dropna().copy()
df_2024_cleaned = df_2024.loc[df_2024_features_cleaned.index].copy()

# 라벨 인코딩 (기존에 학습된 인코더 사용)
# 새로운 값을 처리하기 위해 np.where를 사용해 학습 데이터에 없는 값은 "기타"로 처리
df_2024_features_cleaned['계좌적요'] = np.where(
    df_2024_features_cleaned['계좌적요'].isin(label_encoder_계좌적요.classes_),
    df_2024_features_cleaned['계좌적요'],
    '기타'
)
df_2024_features_cleaned['은행'] = np.where(
    df_2024_features_cleaned['은행'].isin(label_encoder_은행.classes_),
    df_2024_features_cleaned['은행'],
    '기타'
)

# 기존 인코더에 "기타" 값을 추가하여 인코딩
if '기타' not in label_encoder_계좌적요.classes_:
    label_encoder_계좌적요.classes_ = np.append(label_encoder_계좌적요.classes_, '기타')
if '기타' not in label_encoder_은행.classes_:
    label_encoder_은행.classes_ = np.append(label_encoder_은행.classes_, '기타')

# 변환
df_2024_features_cleaned['계좌적요'] = label_encoder_계좌적요.transform(df_2024_features_cleaned['계좌적요'])
df_2024_features_cleaned['은행'] = label_encoder_은행.transform(df_2024_features_cleaned['은행'])

# '용도' 예측 수행
y_2024_pred = voting_clf.predict(df_2024_features_cleaned)

# 예측된 '용도'를 원래 값으로 디코딩
y_2024_pred_original = label_encoder_용도.inverse_transform(y_2024_pred)

# 예측된 '용도'를 원래 데이터에 추가 (결측치 제거된 데이터만 사용)
df_2024_cleaned['예측용도'] = y_2024_pred_original

# 예측 결과를 엑셀로 저장
output_file_2024 = os.path.join(company_result_root, f'{company_name}_입금내역_용도_예측(2024).xlsx')
df_2024_cleaned[['입금일자', '입금액', '계좌적요', '은행', '예측용도']].to_excel(output_file_2024, index=False)
print('2024년 예측 결과가 Excel 파일로 저장되었습니다.')

2024년 예측 결과가 Excel 파일로 저장되었습니다.


#### **2) 입금 파일 : '거래처' Prediction**

In [23]:
# '입금일자'를 datetime 형식으로 변환
df['입금일자'] = pd.to_datetime(df['입금일자'], errors='coerce')

# 필터링할 최종 날짜 설정 (2024년 4월 30일)
end_date = pd.Timestamp('2024-04-30')

# '입금일자'가 end_date 이하인 데이터만 필터링
filtered_df = df[df['입금일자'] <= end_date].copy()

# '거래처'와 '상대계좌예금주명(비고)'이 동일한지 여부를 나타내는 특성 생성
filtered_df['거래처_상대계좌일치'] = filtered_df.apply(
    lambda row: 1 if pd.notna(row['거래처']) and pd.notna(row['비고']) and row['거래처'] in row['비고'] else 0, 
    axis=1
)

# 예측에 사용할 특성 선택
filtered_df_pred = filtered_df[['입금액', '계좌적요', '은행', '용도', '거래처', '거래처_상대계좌일치']].copy()

# 결측치가 있는 행 제거
cleaned_df = filtered_df_pred.dropna().copy()

# 클래스별 데이터 개수 확인
class_counts = cleaned_df['거래처'].value_counts()

# 클래스 수가 2개 미만인 클래스 제거
min_class_count = 2
classes_to_keep = class_counts[class_counts >= min_class_count].index
cleaned_df_filtered = cleaned_df[cleaned_df['거래처'].isin(classes_to_keep)].copy()

# 다시 X와 y 정의
X = cleaned_df_filtered[['입금액', '계좌적요', '거래처_상대계좌일치']].copy()
y = cleaned_df_filtered['거래처'].copy()

# 범주형 변수(Label Encoding) 변환 (클래수 제거 후 적용)
label_encoder_계좌적요 = LabelEncoder()
label_encoder_일치여부 = LabelEncoder()
label_encoder_거래처 = LabelEncoder()

# Fit Transform
X['계좌적요'] = label_encoder_계좌적요.fit_transform(X['계좌적요'].copy())
X['거래처_상대계좌일치'] = label_encoder_일치여부.fit_transform(X['거래처_상대계좌일치'].copy())
y = label_encoder_거래처.fit_transform(y.copy())

# 데이터셋을 stratify를 사용해 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 클래스 불균형 문제를 해결하기 위해 Random Over Sampling 적용
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 오버샘플링한 학습 데이터에서 20%를 검증 데이터로 분리 (eval_set용)
X_train_ros, X_val_ros, y_train_ros, y_val_ros = train_test_split(
    X_train_ros, y_train_ros, test_size=0.2, random_state=42, stratify=y_train_ros
)

# XGBoost 모델 학습 (eval_set을 사용해 성능 모니터링)
xgb_model = XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y_train)), eval_metric='mlogloss')

print('XGBoost Training Progress:')
xgb_model.fit(X_train_ros, y_train_ros, eval_set=[(X_val_ros, y_val_ros)])

# RandomForest 모델 학습 시 tqdm 진행률 표시
n_estimators = 100
rf_model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

print('RandomForest Training Progress:')
for i in tqdm(range(1, n_estimators + 1), desc='RandomForest Training'):
    rf_model.set_params(n_estimators=i)
    rf_model.fit(X_train_ros, y_train_ros)

# Voting Classifier 생성 (XGBoost + RandomForest)
voting_clf = VotingClassifier(estimators=[('xgb', xgb_model), ('rf', rf_model)], voting='soft')

# Voting Classifier 학습 시 tqdm 진행률 표시
print('Voting Classifier Training Progress:')
for _ in tqdm(range(1), desc='Voting Classifier Training'):
    voting_clf.fit(X_train_ros, y_train_ros)

# 테스트 데이터로 예측 수행
y_pred = voting_clf.predict(X_test)

XGBoost Training Progress:
[0]	validation_0-mlogloss:2.21602
[1]	validation_0-mlogloss:1.23352
[2]	validation_0-mlogloss:0.87298
[3]	validation_0-mlogloss:0.64612
[4]	validation_0-mlogloss:0.48912
[5]	validation_0-mlogloss:0.37515
[6]	validation_0-mlogloss:0.29259
[7]	validation_0-mlogloss:0.23157
[8]	validation_0-mlogloss:0.18686
[9]	validation_0-mlogloss:0.15250
[10]	validation_0-mlogloss:0.12679
[11]	validation_0-mlogloss:0.10745
[12]	validation_0-mlogloss:0.09279
[13]	validation_0-mlogloss:0.08165
[14]	validation_0-mlogloss:0.07285
[15]	validation_0-mlogloss:0.06655
[16]	validation_0-mlogloss:0.06240
[17]	validation_0-mlogloss:0.05982
[18]	validation_0-mlogloss:0.05809
[19]	validation_0-mlogloss:0.05690
[20]	validation_0-mlogloss:0.05605
[21]	validation_0-mlogloss:0.05541
[22]	validation_0-mlogloss:0.05491
[23]	validation_0-mlogloss:0.05444
[24]	validation_0-mlogloss:0.05403
[25]	validation_0-mlogloss:0.05370
[26]	validation_0-mlogloss:0.05334
[27]	validation_0-mlogloss:0.05296
[28

RandomForest Training: 100%|██████████| 100/100 [00:26<00:00,  3.77it/s]


Voting Classifier Training Progress:


Voting Classifier Training: 100%|██████████| 1/1 [00:05<00:00,  5.37s/it]


In [24]:
# XGBoost + RandomForest Ensemble Result
# LabelEncoder로 예측된 값을 다시 원래 값으로 디코딩
y_test_original = label_encoder_거래처.inverse_transform(y_test)
y_pred_original = label_encoder_거래처.inverse_transform(y_pred)

# Classification_report를 Dictionary로 변환
report = classification_report(y_test_original, y_pred_original, zero_division=1, output_dict=True)

# Accuracy만 출력
accuracy = round(report['accuracy'], 3)
print(f"Accuracy: {accuracy}")

Accuracy: 0.906


In [25]:
# classification_report를 데이터프레임으로 변환하고 엑셀로 저장하는 간단한 코드
report_df = pd.DataFrame(report).transpose()

# 소수점 3자리로 변환
report_df = report_df.round(3)

# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(1st)')
company_result_root = os.path.join(result_root, company_name)
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

# 결과물 최종 경로
output_file = os.path.join(company_result_root, f'{company_name}_입금내역_거래처.xlsx')

# 엑셀 파일로 저장
report_df.to_excel(output_file, index=True)
print('Classification Report가 Excel 파일로 저장되었습니다.')

Classification Report가 Excel 파일로 저장되었습니다.


In [26]:
# 2024년 Data 예측 #

# '입금일자'를 datetime 형식으로 변환하고 2024년 데이터를 필터링
df_2024 = df[df['입금일자'].dt.year == 2024].copy()

# '거래처'와 '상대계좌예금주명(비고)'이 동일한지 여부를 나타내는 특성 생성
df_2024['거래처_상대계좌일치'] = df_2024.apply(
    lambda row: 1 if pd.notna(row['거래처']) and pd.notna(row['비고']) and row['거래처'] in row['비고'] else 0, 
    axis=1
)

# 예측에 필요한 컬럼만 선택
df_2024_features = df_2024[['입금액', '계좌적요', '거래처_상대계좌일치']].copy()

# 결측치가 있는 행 제거 (원본 데이터에서 인덱스 추적)
df_2024_features_cleaned = df_2024_features.dropna().copy()
df_2024_cleaned = df_2024.loc[df_2024_features_cleaned.index].copy()

# 라벨 인코딩 (기존에 학습된 인코더 사용)
# 새로운 값을 처리하기 위해 np.where를 사용해 학습 데이터에 없는 값은 '기타'로 처리
df_2024_features_cleaned['계좌적요'] = np.where(
    df_2024_features_cleaned['계좌적요'].isin(label_encoder_계좌적요.classes_),
    df_2024_features_cleaned['계좌적요'],
    '기타'
)

df_2024_features_cleaned['거래처_상대계좌일치'] = np.where(
    df_2024_features_cleaned['거래처_상대계좌일치'].isin(label_encoder_일치여부.classes_),
    df_2024_features_cleaned['거래처_상대계좌일치'],
    '기타'
)

# 기존 인코더에 '기타' 값을 추가하여 인코딩
if '기타' not in label_encoder_계좌적요.classes_:
    label_encoder_계좌적요.classes_ = np.append(label_encoder_계좌적요.classes_, '기타')

if '기타' not in label_encoder_일치여부.classes_:
    label_encoder_일치여부.classes_ = np.append(label_encoder_일치여부.classes_, '기타')

# 변환
df_2024_features_cleaned['계좌적요'] = label_encoder_계좌적요.transform(df_2024_features_cleaned['계좌적요'])
df_2024_features_cleaned['거래처_상대계좌일치'] = label_encoder_일치여부.transform(df_2024_features_cleaned['거래처_상대계좌일치'])

# '거래처' 예측 수행
y_2024_pred = voting_clf.predict(df_2024_features_cleaned)

# 예측된 '거래처'를 원래 값으로 디코딩
y_2024_pred_original = label_encoder_거래처.inverse_transform(y_2024_pred)

# 예측된 '거래처'를 원래 데이터에 추가 (결측치 제거된 데이터만 사용)
df_2024_cleaned['예측거래처'] = y_2024_pred_original

# 예측 결과를 엑셀로 저장
output_file_2024 = os.path.join(company_result_root, f'{company_name}_입금내역_거래처_예측(2024).xlsx')
df_2024_cleaned[['입금일자', '입금액', '계좌적요', '거래처_상대계좌일치', '예측거래처']].to_excel(output_file_2024, index=False)
print('2024년 예측 결과가 Excel 파일로 저장되었습니다.')

2024년 예측 결과가 Excel 파일로 저장되었습니다.


---

#### **2. 출금내역 Data Load**

In [28]:
# '출금내역_지급확인' 파일명
file_name = [file for file in os.listdir() if '출금내역_지급확인' in file] # 출금내역 파일 1개

# Data Frame
df = pd.read_excel(file_name[0], skiprows=1)

# Data Frame Head(2023)
df_without_account = df.drop(columns=['계좌번호', '계좌적요', '거래처', '내용'])
df_without_account[df_without_account['출금일자'].str.contains('2023-12')].head()

,출금일자,은행,출금액,용도,프로젝트/현장,비고,메모
181,2023-12-29,우리은행,993800,매입대금지급,NaN,결의서_202312_19\n2023-12-28 세계 수수료 500,NaN
182,2023-12-29,우리은행,880500,매입대금지급,NaN,결의서_202312_18\n2023-12-27 세계 수수료 500,NaN
183,2023-12-29,우리은행,286500,매입대금지급,NaN,결의서_202312_17\n2023-12-26 세계 수수료 500,NaN
184,2023-12-29,우리은행,195560,매입대금지급,NaN,결의서_202312_16\n결의서_202312_16\n2023-12-18 세계202...,NaN
185,2023-12-27,우리은행,200000,일반경조사비,NaN,NaN,NaN


#### **1) 출금 파일 : '용도' Prediction**

In [29]:
# '출금일자'를 datetime 형식으로 변환하고 2023년까지 필터링
df['출금일자'] = pd.to_datetime(df['출금일자'], errors='coerce')
df_filtered = df[df['출금일자'].dt.year <= 2023].copy()

# '용도'와 '거래처'의 결측치가 있는 행 제거
df_filtered_cleaned = df_filtered.dropna(subset=['용도', '거래처']).copy()

# 예측에 사용할 특성 선택
df_features = df_filtered_cleaned[['출금액', '계좌적요', '은행', '용도', '거래처']].copy()

# 결측치가 있는 행 제거
df_features_cleaned = df_features.dropna().copy()

# 클래스별 데이터 개수 확인
class_counts = df_features_cleaned['용도'].value_counts()

# 클래스 수가 2개 미만인 클래스 제거
min_class_count = 2
classes_to_keep = class_counts[class_counts >= min_class_count].index
df_filtered_cleaned = df_features_cleaned[df_features_cleaned['용도'].isin(classes_to_keep)].copy()

# 다시 X와 y 정의
X = df_filtered_cleaned[['출금액', '계좌적요', '은행']].copy()
y = df_filtered_cleaned['용도'].copy()

# 범주형 변수(Label Encoding) 변환 (클래스 제거 후 적용)
label_encoder_계좌적요 = LabelEncoder()
label_encoder_은행 = LabelEncoder()
label_encoder_용도 = LabelEncoder()

# Fit Transform
X['계좌적요'] = label_encoder_계좌적요.fit_transform(X['계좌적요'].copy())
X['은행'] = label_encoder_은행.fit_transform(X['은행'].copy())
y = label_encoder_용도.fit_transform(y.copy())

# 데이터셋을 stratify를 사용해 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 클래스 불균형 문제를 해결하기 위해 Random Over Sampling 적용
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 오버샘플링한 학습 데이터에서 20%를 검증 데이터로 분리 (eval_set용)
X_train_ros, X_val_ros, y_train_ros, y_val_ros = train_test_split(
    X_train_ros, y_train_ros, test_size=0.2, random_state=42, stratify=y_train_ros
)

# XGBoost 모델 학습 (eval_set을 사용해 성능 모니터링)
xgb_model = XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y_train)), eval_metric='mlogloss')

print('XGBoost Training Progress:')
xgb_model.fit(X_train_ros, y_train_ros, eval_set=[(X_val_ros, y_val_ros)])

# RandomForest 모델 학습 시 tqdm 진행률 표시
n_estimators = 100
rf_model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

print('RandomForest Training Progress:')
for i in tqdm(range(1, n_estimators + 1), desc='RandomForest Training'):
    rf_model.set_params(n_estimators=i)
    rf_model.fit(X_train_ros, y_train_ros)

# Voting Classifier 생성 (XGBoost + RandomForest)
voting_clf = VotingClassifier(estimators=[('xgb', xgb_model), ('rf', rf_model)], voting='soft')

# Voting Classifier 학습 시 tqdm 진행률 표시
print('Voting Classifier Training Progress:')
for _ in tqdm(range(1), desc='Voting Classifier Training'):
    voting_clf.fit(X_train_ros, y_train_ros)

# 테스트 데이터로 예측 수행
y_pred = voting_clf.predict(X_test)

XGBoost Training Progress:
[0]	validation_0-mlogloss:0.81608
[1]	validation_0-mlogloss:0.58970
[2]	validation_0-mlogloss:0.45281
[3]	validation_0-mlogloss:0.35570
[4]	validation_0-mlogloss:0.28036
[5]	validation_0-mlogloss:0.22655
[6]	validation_0-mlogloss:0.18822
[7]	validation_0-mlogloss:0.15922
[8]	validation_0-mlogloss:0.13632
[9]	validation_0-mlogloss:0.11910
[10]	validation_0-mlogloss:0.10564
[11]	validation_0-mlogloss:0.09606
[12]	validation_0-mlogloss:0.08823
[13]	validation_0-mlogloss:0.08198
[14]	validation_0-mlogloss:0.07689
[15]	validation_0-mlogloss:0.07299
[16]	validation_0-mlogloss:0.06999
[17]	validation_0-mlogloss:0.06764
[18]	validation_0-mlogloss:0.06547
[19]	validation_0-mlogloss:0.06398
[20]	validation_0-mlogloss:0.06276
[21]	validation_0-mlogloss:0.06176
[22]	validation_0-mlogloss:0.06091
[23]	validation_0-mlogloss:0.06022
[24]	validation_0-mlogloss:0.05960
[25]	validation_0-mlogloss:0.05911
[26]	validation_0-mlogloss:0.05864
[27]	validation_0-mlogloss:0.05840
[28

RandomForest Training: 100%|██████████| 100/100 [00:15<00:00,  6.25it/s]


Voting Classifier Training Progress:


Voting Classifier Training: 100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


In [30]:
# XGBoost + RandomForest Ensemble Result
# LabelEncoder로 예측된 값을 다시 원래 값으로 디코딩
y_test_original = label_encoder_용도.inverse_transform(y_test)
y_pred_original = label_encoder_용도.inverse_transform(y_pred)

# Classification_report를 Dictionary로 변환
report = classification_report(y_test_original, y_pred_original, zero_division=1, output_dict=True)

# Accuracy만 출력
accuracy = round(report['accuracy'], 3)
print(f"Accuracy: {accuracy}")

Accuracy: 0.927


In [31]:
# classification_report를 데이터프레임으로 변환하고 엑셀로 저장하는 간단한 코드
report_df = pd.DataFrame(report).transpose()

# 소수점 3자리로 변환
report_df = report_df.round(3)

# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(1st)')
company_result_root = os.path.join(result_root, company_name)
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

# 결과물 최종 경로
output_file = os.path.join(company_result_root, f'{company_name}_출금내역_용도.xlsx')

# 엑셀 파일로 저장
report_df.to_excel(output_file, index=True)
print('Classification Report가 Excel 파일로 저장되었습니다.')

Classification Report가 Excel 파일로 저장되었습니다.


In [32]:
# 2024년 Data 예측 #

# '출금일자'를 datetime 형식으로 변환하고 2024년 필터링
df_2024 = df[df['출금일자'].dt.year == 2024].copy()

# 예측에 필요한 컬럼만 선택
df_2024_features = df_2024[['출금액', '계좌적요', '은행']].copy()

# 결측치가 있는 행 제거 (원본 데이터에서 인덱스 추적)
df_2024_features_cleaned = df_2024_features.dropna().copy()
df_2024_cleaned = df_2024.loc[df_2024_features_cleaned.index].copy()

# 라벨 인코딩 (기존에 학습된 인코더 사용)
# 새로운 값을 처리하기 위해 np.where를 사용해 학습 데이터에 없는 값은 "기타"로 처리
df_2024_features_cleaned['계좌적요'] = np.where(
    df_2024_features_cleaned['계좌적요'].isin(label_encoder_계좌적요.classes_),
    df_2024_features_cleaned['계좌적요'],
    '기타'
)
df_2024_features_cleaned['은행'] = np.where(
    df_2024_features_cleaned['은행'].isin(label_encoder_은행.classes_),
    df_2024_features_cleaned['은행'],
    '기타'
)

# 기존 인코더에 "기타" 값을 추가하여 인코딩
if '기타' not in label_encoder_계좌적요.classes_:
    label_encoder_계좌적요.classes_ = np.append(label_encoder_계좌적요.classes_, '기타')
if '기타' not in label_encoder_은행.classes_:
    label_encoder_은행.classes_ = np.append(label_encoder_은행.classes_, '기타')

# 변환
df_2024_features_cleaned['계좌적요'] = label_encoder_계좌적요.transform(df_2024_features_cleaned['계좌적요'])
df_2024_features_cleaned['은행'] = label_encoder_은행.transform(df_2024_features_cleaned['은행'])

# '용도' 예측 수행
y_2024_pred = voting_clf.predict(df_2024_features_cleaned)

# 예측된 '용도'를 원래 값으로 디코딩
y_2024_pred_original = label_encoder_용도.inverse_transform(y_2024_pred)

# 예측된 '용도'를 원래 데이터에 추가 (결측치 제거된 데이터만 사용)
df_2024_cleaned['예측용도'] = y_2024_pred_original

# 예측 결과를 엑셀로 저장
output_file_2024 = os.path.join(company_result_root, f'{company_name}_출금내역_용도_예측(2024).xlsx')
df_2024_cleaned[['출금일자', '출금액', '계좌적요', '은행', '예측용도']].to_excel(output_file_2024, index=False)
print('2024년 예측 결과가 Excel 파일로 저장되었습니다.')

2024년 예측 결과가 Excel 파일로 저장되었습니다.


#### **2) 출금 파일 : '거래처' Prediction**

In [33]:
# '출금일자'를 datetime 형식으로 변환하고 2023년까지 필터링
df['출금일자'] = pd.to_datetime(df['출금일자'], errors='coerce')
filtered_df = df[df['출금일자'].dt.year <= 2023].copy()

# '거래처'와 '상대계좌예금주명(비고)'이 동일한지 여부를 나타내는 특성 생성
filtered_df['거래처_상대계좌일치'] = filtered_df.apply(
    lambda row: 1 if pd.notna(row['거래처']) and pd.notna(row['비고']) and row['거래처'] in row['비고'] else 0, 
    axis=1
)

# 예측에 사용할 특성 선택
filtered_df_pred = filtered_df[['출금액', '계좌적요', '은행', '용도', '거래처', '거래처_상대계좌일치']].copy()

# 결측치가 있는 행 제거
cleaned_df = filtered_df_pred.dropna().copy()

# 클래스별 데이터 개수 확인
class_counts = cleaned_df['거래처'].value_counts()

# 클래스 수가 2개 미만인 클래스 제거
min_class_count = 2
classes_to_keep = class_counts[class_counts >= min_class_count].index
cleaned_df_filtered = cleaned_df[cleaned_df['거래처'].isin(classes_to_keep)].copy()

# 다시 X와 y 정의
X = cleaned_df_filtered[['출금액', '계좌적요', '거래처_상대계좌일치']].copy()
y = cleaned_df_filtered['거래처'].copy()

# 범주형 변수(Label Encoding) 변환 (클래수 제거 후 적용)
label_encoder_계좌적요 = LabelEncoder()
label_encoder_일치여부 = LabelEncoder()
label_encoder_거래처 = LabelEncoder()

# Fit Transform
X['계좌적요'] = label_encoder_계좌적요.fit_transform(X['계좌적요'].copy())
X['거래처_상대계좌일치'] = label_encoder_일치여부.fit_transform(X['거래처_상대계좌일치'].copy())
y = label_encoder_거래처.fit_transform(y.copy())

# 데이터셋을 stratify를 사용해 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 클래스 불균형 문제를 해결하기 위해 Random Over Sampling 적용
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# 오버샘플링한 학습 데이터에서 20%를 검증 데이터로 분리 (eval_set용)
X_train_ros, X_val_ros, y_train_ros, y_val_ros = train_test_split(
    X_train_ros, y_train_ros, test_size=0.2, random_state=42, stratify=y_train_ros
)

# XGBoost 모델 학습 (eval_set을 사용해 성능 모니터링)
xgb_model = XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y_train)), eval_metric='mlogloss')

print('XGBoost Training Progress:')
xgb_model.fit(X_train_ros, y_train_ros, eval_set=[(X_val_ros, y_val_ros)])

# RandomForest 모델 학습 시 tqdm 진행률 표시
n_estimators = 100
rf_model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

print('RandomForest Training Progress:')
for i in tqdm(range(1, n_estimators + 1), desc='RandomForest Training'):
    rf_model.set_params(n_estimators=i)
    rf_model.fit(X_train_ros, y_train_ros)

# Voting Classifier 생성 (XGBoost + RandomForest)
voting_clf = VotingClassifier(estimators=[('xgb', xgb_model), ('rf', rf_model)], voting='soft')

# Voting Classifier 학습 시 tqdm 진행률 표시
print('Voting Classifier Training Progress:')
for _ in tqdm(range(1), desc='Voting Classifier Training'):
    voting_clf.fit(X_train_ros, y_train_ros)

# 테스트 데이터로 예측 수행
y_pred = voting_clf.predict(X_test)

XGBoost Training Progress:
[0]	validation_0-mlogloss:1.02428
[1]	validation_0-mlogloss:0.75184
[2]	validation_0-mlogloss:0.58615
[3]	validation_0-mlogloss:0.47362
[4]	validation_0-mlogloss:0.39290
[5]	validation_0-mlogloss:0.33351
[6]	validation_0-mlogloss:0.29004
[7]	validation_0-mlogloss:0.25657
[8]	validation_0-mlogloss:0.23201
[9]	validation_0-mlogloss:0.21277
[10]	validation_0-mlogloss:0.19803
[11]	validation_0-mlogloss:0.18676
[12]	validation_0-mlogloss:0.17778
[13]	validation_0-mlogloss:0.17091
[14]	validation_0-mlogloss:0.16562
[15]	validation_0-mlogloss:0.16114
[16]	validation_0-mlogloss:0.15740
[17]	validation_0-mlogloss:0.15464
[18]	validation_0-mlogloss:0.15217
[19]	validation_0-mlogloss:0.15027
[20]	validation_0-mlogloss:0.14843
[21]	validation_0-mlogloss:0.14694
[22]	validation_0-mlogloss:0.14593
[23]	validation_0-mlogloss:0.14501
[24]	validation_0-mlogloss:0.14430
[25]	validation_0-mlogloss:0.14366
[26]	validation_0-mlogloss:0.14318
[27]	validation_0-mlogloss:0.14276
[28

RandomForest Training: 100%|██████████| 100/100 [00:12<00:00,  8.12it/s]


Voting Classifier Training Progress:


Voting Classifier Training: 100%|██████████| 1/1 [00:01<00:00,  1.95s/it]


In [34]:
# XGBoost + RandomForest Ensemble Result
# LabelEncoder로 예측된 값을 다시 원래 값으로 디코딩
y_test_original = label_encoder_거래처.inverse_transform(y_test)
y_pred_original = label_encoder_거래처.inverse_transform(y_pred)

# Classification_report를 Dictionary로 변환
report = classification_report(y_test_original, y_pred_original, zero_division=1, output_dict=True)

# Accuracy만 출력
accuracy = round(report['accuracy'], 3)
print(f"Accuracy: {accuracy}")

Accuracy: 0.858


In [35]:
# classification_report를 데이터프레임으로 변환하고 엑셀로 저장하는 간단한 코드
report_df = pd.DataFrame(report).transpose()

# 소수점 3자리로 변환
report_df = report_df.round(3)

# 회사명에 따른 결과물 저장 경로
result_root = os.path.join(root, 'Model(1st)')
company_result_root = os.path.join(result_root, company_name)
if not os.path.isdir(company_result_root):
    os.makedirs(company_result_root)

# 결과물 최종 경로
output_file = os.path.join(company_result_root, f'{company_name}_출금내역_거래처.xlsx')

# 엑셀 파일로 저장
report_df.to_excel(output_file, index=True)
print('Classification Report가 Excel 파일로 저장되었습니다.')

Classification Report가 Excel 파일로 저장되었습니다.


In [36]:
# 2024년 Data 예측 #

# '출금일자'를 datetime 형식으로 변환하고 2024년 데이터를 필터링
df_2024 = df[df['출금일자'].dt.year == 2024].copy()

# '거래처'와 '상대계좌예금주명(비고)'이 동일한지 여부를 나타내는 특성 생성
df_2024['거래처_상대계좌일치'] = df_2024.apply(
    lambda row: 1 if pd.notna(row['거래처']) and pd.notna(row['비고']) and row['거래처'] in row['비고'] else 0, 
    axis=1
)

# 예측에 필요한 컬럼만 선택
df_2024_features = df_2024[['출금액', '계좌적요', '거래처_상대계좌일치']].copy()

# 결측치가 있는 행 제거 (원본 데이터에서 인덱스 추적)
df_2024_features_cleaned = df_2024_features.dropna().copy()
df_2024_cleaned = df_2024.loc[df_2024_features_cleaned.index].copy()

# 라벨 인코딩 (기존에 학습된 인코더 사용)
# 새로운 값을 처리하기 위해 np.where를 사용해 학습 데이터에 없는 값은 '기타'로 처리
df_2024_features_cleaned['계좌적요'] = np.where(
    df_2024_features_cleaned['계좌적요'].isin(label_encoder_계좌적요.classes_),
    df_2024_features_cleaned['계좌적요'],
    '기타'
)

df_2024_features_cleaned['거래처_상대계좌일치'] = np.where(
    df_2024_features_cleaned['거래처_상대계좌일치'].isin(label_encoder_일치여부.classes_),
    df_2024_features_cleaned['거래처_상대계좌일치'],
    '기타'
)

# 기존 인코더에 '기타' 값을 추가하여 인코딩
if '기타' not in label_encoder_계좌적요.classes_:
    label_encoder_계좌적요.classes_ = np.append(label_encoder_계좌적요.classes_, '기타')

if '기타' not in label_encoder_일치여부.classes_:
    label_encoder_일치여부.classes_ = np.append(label_encoder_일치여부.classes_, '기타')

# 변환
df_2024_features_cleaned['계좌적요'] = label_encoder_계좌적요.transform(df_2024_features_cleaned['계좌적요'])
df_2024_features_cleaned['거래처_상대계좌일치'] = label_encoder_일치여부.transform(df_2024_features_cleaned['거래처_상대계좌일치'])

# '거래처' 예측 수행
y_2024_pred = voting_clf.predict(df_2024_features_cleaned)

# 예측된 '거래처'를 원래 값으로 디코딩
y_2024_pred_original = label_encoder_거래처.inverse_transform(y_2024_pred)

# 예측된 '거래처'를 원래 데이터에 추가 (결측치 제거된 데이터만 사용)
df_2024_cleaned['예측거래처'] = y_2024_pred_original

# 예측 결과를 엑셀로 저장
output_file_2024 = os.path.join(company_result_root, f'{company_name}_출금내역_거래처_예측(2024).xlsx')
df_2024_cleaned[['출금일자', '출금액', '계좌적요', '거래처_상대계좌일치', '예측거래처']].to_excel(output_file_2024, index=False)
print('2024년 예측 결과가 Excel 파일로 저장되었습니다.')

2024년 예측 결과가 Excel 파일로 저장되었습니다.
